In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import os
import sys
tfds.disable_progress_bar()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile('/tmp/param.txt', 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01))
  return model

NUM_WORKERS = 1
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=5, batch_size=GLOBAL_BATCH_SIZE)
model.save('/home/jesusarguelles/code/vertex-gpu/pipe_notebook/models')

Python Version = 3.7.13 (default, Mar 29 2022, 02:18:16) 
[GCC 7.5.0]
TensorFlow Version = 2.9.1
TF_CONFIG = Not found
Epoch 1/5


/home/jesusarguelles/miniconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


26/26 [==============================] - 1s 2ms/step - loss: 28100.8125
Epoch 2/5
26/26 [==============================] - 0s 2ms/step - loss: 694.7755
Epoch 3/5
26/26 [==============================] - 0s 2ms/step - loss: 677.9030
Epoch 4/5
26/26 [==============================] - 0s 2ms/step - loss: 238.4257
Epoch 5/5
26/26 [==============================] - 0s 2ms/step - loss: 135.5316
INFO:tensorflow:Assets written to: /home/jesusarguelles/code/vertex-gpu/pipe_notebook/models/assets


In [9]:
model.predict(x_train)[-1]

13/13 [==============================] - 0s 774us/step


array([17.222773], dtype=float32)

In [10]:
y_train[-1]

29.1

In [14]:
import numpy as np

mse=np.sum(np.power(y_train-model.predict(x_train).reshape(x_train.shape[0],),2))/y_train.shape[0]
mse

13/13 [==============================] - 0s 578us/step


113.73742764049464

In [16]:
loaded = tf.saved_model.load('models/')

In [23]:
list(loaded.signatures['serving_default'].structured_input_signature[1].keys())[0]

'dense_3_input'

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               1792      
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 18,433
Trainable params: 18,433
Non-trainable params: 0
_________________________________________________________________


In [46]:
list(loaded.signatures['serving_default'].structured_input_signature[1].keys())[0]
serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

Serving function input: dense_3_input
Serving function output: dense_5


In [48]:
input_name = model.input.name
print("Model input name:", input_name)
output_name = model.output.name
print("Model output name:", output_name)

Model input name: dense_3_input
Model output name: dense_5/BiasAdd:0


In [49]:
model

In [50]:
loaded

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7fe89ba9b510>

In [56]:
model.weights

[<tf.Variable 'dense_3/kernel:0' shape=(13, 128) dtype=float32, numpy=
 array([[-0.03077333, -0.23708047, -0.21608256, ...,  0.0849395 ,
          0.08794362, -0.13315007],
        [-0.07570806, -0.14996772, -0.11492363, ..., -0.14895344,
          0.10518547,  0.00861384],
        [ 0.01414169,  0.09263313, -0.30166024, ..., -0.08057518,
         -0.02716043, -0.06649391],
        ...,
        [ 0.17000103, -0.11387086, -0.24519397, ..., -0.01828994,
          0.07408316,  0.00280151],
        [-0.05510873, -0.22886766, -0.16724998, ...,  0.10359433,
          0.09101723, -0.12015251],
        [-0.20242287, -0.19052926, -0.32151076, ..., -0.28676498,
          0.07626975, -0.16386399]], dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(128,) dtype=float32, numpy=
 array([ 0.        , -0.03162277, -0.1164071 , -0.1208004 , -0.0562485 ,
        -0.10173824,  0.01692398, -0.09291069, -0.05122419, -0.07119531,
        -0.04242091, -0.03482908, -0.03199682,  0.        , -0.01233639,
 